In [45]:
import os
import gc
# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/gdrive/projects/programming/automation")
from time import sleep
import sistemas
import sei
import functions
from page import *

% reload_ext autoreload
% autoreload 2

In [46]:
PATH = os.path.abspath(r"C:/Users/rsilva/Desktop/SEI/")
print(PATH)               

C:\Users\rsilva\Desktop\SEI


In [69]:
keys = ('Nome/Razão Social', 'Logradouro', 'Número', 'Complemento', 'Bairro', 'Cep', 'Município', 'UF', 'Número Fistel')

In [74]:
def string_endereço(dados):
    
    s = 'À<br>' 
    
    s += dados["Nome/Razão Social"].title()
    
    s += '<br>' + dados["Logradouro"].title() + ", " + dados["Número"] + " " 
    
    s += dados["Complemento"].title() + " " 
    
    s += dados["Bairro"].title() + '<br>' 
    
    s += "CEP: " + dados["Cep"] + " - " + dados["Município"].title() + " - " + dados["UF"] 
            
    s += "<br><br>" + "<b>FISTEL: " + dados["Número Fistel"] + "</b>"
            
    return s

In [90]:
usr = "rsilva"
pwd = "Savorthemom3nts"
browser = webdriver.Firefox()

In [91]:
sei_ = sei.login_sei(browser, usr, pwd)

In [92]:
sei_ = sei.Sei(sei_)

In [61]:
sei_.itera_processos()

In [17]:
#scpx = webdriver.Ie()
scpx = sistemas.Scpx(scpx.driver)

In [62]:
cpf = '14804020870'
proc = r'53504.008508/2018-89'

In [93]:
p = sei_.go_to_processo(proc)

In [18]:
scpx.consulta(cpf)

In [36]:
scpx.incluir_estacao(cpf, 'Móvel', 'PX2J1510')

In [37]:
scpx.movimento_transferir(cpf, 'B', 'E', proc=proc)

In [38]:
scpx.licenciar_estacao(cpf)

In [39]:
scpx.licenciar_estacao(cpf, ppdess=False)

In [40]:
files = sorted(os.listdir(PATH), reverse=True)
file = os.path.join(PATH, files[0])

In [47]:
p.incluir_doc_externo('Boleto', file)

In [65]:
dados = scpx.extrai_cadastro(cpf)

In [71]:
endereco = {k:v for k, v in dados.items() if k in keys}

In [89]:
p.incluir_oficio('RC_OF_PPDESS', string_endereço(endereco))

NoAlertPresentException: Message: No modal dialog is currently open


In [39]:
string.format(tag)

"//p[@class='Texto_Alinhado_Esquerda' and contains(text(), '30')]"